<a href="https://colab.research.google.com/github/Han-Daon/Project/blob/main/P_median_%EC%A0%81%EC%9A%A9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
pip install haversine

In [4]:
import numpy as np
import pandas as pd
from haversine import haversine

In [7]:
입지후보지 = pd.read_csv('행정경계_중심.csv', encoding='cp949')
후보지_points = np.array([list(i) for i in zip(입지후보지['y'], 입지후보지['x'])])

보건소 = pd.read_csv('보건소_위도경도.csv', encoding='cp949')
보건소_points = np.array([list(i) for i in zip(보건소['V3'], 보건소['V4'])])

영화관 = pd.read_csv('영화관_위도경도.csv', encoding='cp949')
영화관_points = np.array([list(i) for i in zip(영화관['V3'], 영화관['V4'])])

도시공원 = pd.read_csv('전라북도 도시공원.csv', encoding='cp949')
도시공원_points = np.array([list(i) for i in zip(도시공원['위도'], 도시공원['경도'])])

행정기관 = pd.read_csv('행정기관_위도경도.csv',encoding='cp949')
행정기관_points = np.array([list(i) for i in zip(행정기관['V3'], 행정기관['V4'])])

In [8]:
X = list(행정기관['V4']) + list(영환관['V4']) + list(보건소['V4']) + list(도시공원['경도'])
Y = list(도시공원['위도']) + list(행정기관['V3']) + list(보건소['V3']) + list(영환관['V3'])
points = np.array([list(i) for i in zip(X, Y)])
print(points.shape)
points[:2]

(1353, 2)


array([[127.1406569 ,  35.72771983],
       [127.1480759 ,  35.6411154 ]])

In [10]:
전체w = points.shape[0]
행정기관w = 행정기관.shape[0]
보건소w = 보건소.shape[0]
영화관w = 영화관.shape[0]
도시공원w = 도시공원.shape[0]

# 선호도에 대한 가중치 * 데이터 불균형 해소를 위한 가중치
# 데이터 불균형 해소를 위한 가중치는 (전체수요지점수 - 해당수요지점수) / 전체수요지점수

m1 = 0.278 * (전체w-행정기관w) / 전체w
m2 = 0.193 * (전체w-보건소w) / 전체w
m3 = 0.136 * (전체w-영화관w) / 전체w
m4 = 0.392 * (전체w-도시공원w) / 전체w

In [13]:
def pmedian(후보지_points, 보건소_points, 영화관_points, 도시공원_points, 행정기관_points):


    havers1 = []
    for i in 후보지_points:
        site1 = []
        for j in 행정기관_points:
            site1.append(haversine(i,j)*1000)
        havers1.append(site1)

    location = list(입지후보지['EMD_NM'])
    location1 = list(행정기관['읍면동'])

    havers_D1 = dict(zip(location, [dict(zip(location1, i)) for i in havers1]))
    D1 = pd.DataFrame(havers_D1)

    최소값 = list(D1.min(axis=1))

    # 최단거리인 경우 가중치 부여, 아니면 0
    idx = D1.index
    col = D1.columns
    lidx = len(idx)
    lcol = len(col)

    for i in range(lidx):
        for j in range(lcol):
            if D1.loc[idx[i], col[j]] == 최소값[i]:
                D1.loc[idx[i], col[j]] = m1
            else:
                D1.loc[idx[i], col[j]] = 0



    havers2 = []
    for i in 후보지_points:
        site2 = []
        for j in 보건소_points:
            site2.append(haversine(i,j)*1000)
        havers2.append(site2)

    location2 = list(보건소['보건소명'])

    havers_D2 = dict(zip(location, [dict(zip(location2, i)) for i in havers2]))
    D2 = pd.DataFrame(havers_D2)

    최소값 = list(D2.min(axis=1))

    idx = D2.index
    col = D2.columns
    lidx = len(idx)
    lcol = len(col)

    for i in range(lidx):
        for j in range(lcol):
            if D2.loc[idx[i], col[j]] == 최소값[i]:
                D2.loc[idx[i], col[j]] = m2
            else:
                D2.loc[idx[i], col[j]] = 0



    havers3 = []
    for i in 후보지_points:
        site3 = []
        for j in 영화관_points:
            site3.append(haversine(i,j)*1000)
        havers3.append(site3)

    location3 = list(영화관['영화상영관명'])

    havers_D3 = dict(zip(location, [dict(zip(location3, i)) for i in havers3]))
    D3 = pd.DataFrame(havers_D3)

    최소값 = list(D3.min(axis=1))

    idx = D3.index
    col = D3.columns
    lidx = len(idx)
    lcol = len(col)

    for i in range(lidx):
        for j in range(lcol):
            if D3.loc[idx[i], col[j]] == 최소값[i]:
                D3.loc[idx[i], col[j]] = m3
            else:
                D3.loc[idx[i], col[j]] = 0



    havers4 = []
    for i in 후보지_points:
        site4 = []
        for j in 도시공원_points:
            site4.append(haversine(i,j)*1000)
        havers4.append(site4)

    location4 = list(도시공원['공원명'])

    havers_D4 = dict(zip(location, [dict(zip(location4, i)) for i in havers4]))
    D4 = pd.DataFrame(havers_D4)

    최소값 = list(D4.min(axis=1))

    idx = D4.index
    col = D4.columns
    lidx = len(idx)
    lcol = len(col)

    for i in range(lidx):
        for j in range(lcol):
            if D4.loc[idx[i], col[j]] == 최소값[i]:
                D4.loc[idx[i], col[j]] = m4
            else:
                D4.loc[idx[i], col[j]] = 0

    D_final = pd.concat([D1, D2, D3, D4])
    print(D_final.sum().sort_values(ascending=False))



In [14]:
df = pmedian(후보지_points, 보건소_points, 도시공원_points, 영화관_points, 행정기관_points)

도통동    1.425836
장계면    1.262084
진안읍    1.092078
나운동    1.077661
신덕면    0.995914
         ...   
신용동    0.000000
금강동    0.000000
석탄동    0.000000
덕기동    0.000000
농소동    0.000000
Length: 388, dtype: float64


In [15]:
dff = pd.read_csv('지역+중요도.csv', encoding='cp949')

In [16]:
dff.sort_values('중요도', ascending=False)

,지역,중요도
7,성내면,19.990355
33,부남면,18.043266
35,산서면,15.396314
5,쌍치면,14.992627
22,덕과면,14.188708
30,성수면,11.856437
3,신덕면,11.824898
8,백산면,11.192170
15,감곡면,8.935960
28,이서면,7.345283
